# 추천 시스템

## 1. 콘텐츠 기반 필터링 추천 시스템

### (1) 데이터 로딩

In [12]:
import pandas as pd 
import numpy as np
import warnings; warnings.filterwarnings('ignore')
movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


#### 1) 필요 컬럼을 추출해서 데이터프레임 생성

In [13]:
movies_df = movies[['id', 'title', 'genres','vote_average','vote_count','popularity','keywords','overview']]

In [14]:
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster..."


In [15]:
# genres, keywords 컬럼 내용 확인
pd.set_option('max_colwidth',100)
movies_df[['genres', 'keywords']][:2]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is..."


In [16]:

# literal_eval() 함수를 사용하여 문자열을 원래의 데이터 타입으로 변환
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)from ast import literal_eval

#### 이제 genres, keywords 컬럼은 문자열이 아닌 딕셔너리 객체로 변환되었음.

In [17]:
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."


#### 2) 'genres','keywords' 컬럼에서 'name'키에 해당하는 값을 찾아 리스트로 추출하여 저장

In [18]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### (2) movies_df 데이터프레임 저장

In [19]:
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."


In [20]:
# pickle 타입으로 데이터 저장
import pickle
movies_df.to_pickle('movies_df_pkl')

### (3) 장르별 콘텐츠 유사도 측정

#### 유사도 측정 방식
#### doc1 = 'She likes python'
#### doc2 = 'She hates python'
#### CountVectorizer 클래스를 이용하여 각 문서 내의 단어 빈도수 계산

In [21]:
#               She     likes       hates      python
# --------------------------------------------------------
#    doc1        1        1           0            1
#    doc2        1        0           1           1

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0., ngram_range=(1,2))  # min_df - 최소 데이터 빈도, ngram_range - 단어의 묶음을 1개 또는 2개로 지정.
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


#### 사이킷런의 cosine_similaritry() 를 이용해 코사인 유사도 계산

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [35]:
# 0번 레코드의 유사도 인덱스 값 추출
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_index)

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]
 [   2 1740 1542 ... 3000 2999 2401]
 ...
 [4800 3809 1895 ... 2229 2230    0]
 [4802 1594 1596 ... 3204 3205    0]
 [4802 4710 4521 ... 3140 3141    0]]


#### 0번 레코드는 자신을 제외하고 id가 3494인 영화가 가장 유사도가 높음.

### (4) 장르 유사도에 따라 영화를 추천하는 함수 작성

In [36]:
# 매개변수 : df - movies_df, sorted_ind- 유사도 측정 matrix, title_name - 영화제목, top_n - 추천 영화 수
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 매개변수로 입력된 title_name의 영화정보를 읽는다.
    title_movie = df[df['title'] == title_name]

    # 추출한 title_name의 영화정보에서 id를 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    print(similar_indexes)
    # 추출된 벡터를 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

#### find_sim_movie()함수를 사용해 영화'대부'와 장르별로 유사한 영화 추천

In [38]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['id','title','vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,id,title,vote_average
2731,240,The Godfather: Part II,8.3
1243,203,Mean Streets,7.2
3636,36351,Light Sleeper,5.7
1946,11699,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,400,Things to Do in Denver When You're Dead,6.7
4065,364083,Mi America,0.0
1847,769,GoodFellas,8.2
4217,9344,Kids,6.8
883,640,Catch Me If You Can,7.7
3866,598,City of God,8.1


### (5) 영화 평점에 따라 필터링하여 최종 추천하는 방식으로 수정

##### 1) 1~2명의 소수 관객의 높은 영화평점을 부여한 왜곡된 데이터를 보완

In [40]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


#### 쇼생크 탈출과 같은 높은 순위에 이름이 없는 'Stiff Upper Lips'와 같이 영화가 랭크됨

### => 가중 평점 공식을 사용하여 새로운 평점 부여

#### 가중 평점 공식: (v / (v+m)) * R + (m / (v+m)) * C
#### v: 개별 영화에 평점을 투표한 횟수  (vote_count)
#### m: 최소 투표 횟수 (전체 투표 횟수의 60%값 사용)
#### R: 개별 영화 평균 평점 (vote_average)
#### C: 전체 영화의 평균 평점 (movies_df['vote_average'].mean())

In [42]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6) # 사분위 60% 값
print( 'C = ', round(C,3),'m = ', round(m,3)) 

C =  6.092 m =  370.2


In [47]:
# 가중 평점 계산
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

# 특정 영화에 대한 가중평점 계산
def weighted_vote_average(row):
    v = row['vote_count']
    R = row['vote_average']
    
    return ( (v/(v+m)) * R) + ((m/(v+m)) * C)

In [48]:
# 가중평점 컬럼 추가
movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

In [54]:
movies_df.head(3)
movies_df.to_pickle('movies_df.pkl')

In [55]:
movies_df[['id','title','vote_average','vote_count','weighted_vote']].sort_values('weighted_vote', ascending=False)[:10]

,id,title,vote_average,vote_count,weighted_vote
1881,278,The Shawshank Redemption,8.5,8205,8.396052
3337,238,The Godfather,8.4,5893,8.263591
662,550,Fight Club,8.3,9413,8.216455
3232,680,Pulp Fiction,8.3,8428,8.207102
65,155,The Dark Knight,8.2,12002,8.136930
1818,424,Schindler's List,8.3,4329,8.126069
3865,244786,Whiplash,8.3,4254,8.123248
809,13,Forrest Gump,8.2,7927,8.105954
2294,129,Spirited Away,8.3,3840,8.105867
2731,240,The Godfather: Part II,8.3,3338,8.079586


### (5) 새롭게 정의된 평점 기준에 따라 영화 추천

In [56]:
# 장르 유사성이 높은 top_n의 2배 만큼 선정한 뒤, 가중평점이 높은 순으로 top_n만큼 추천
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 매개변수로 입력된 title_name의 영화정보를 읽는다.
    title_movie = df[df['title'] == title_name]

    # 추출한 title_name의 영화정보에서 id를 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    
    # 추출된 벡터를 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)
    
    #기준 영화 인덱스 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    

    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

#### 새로 정의한 함수 테스트

In [57]:
similar_movie = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movie[['id','title','vote_average','vote_count','weighted_vote']]

,id,title,vote_average,vote_count,weighted_vote
2731,240,The Godfather: Part II,8.3,3338,8.079586
1847,769,GoodFellas,8.2,3128,7.976937
3866,598,City of God,8.1,1814,7.759693
1663,311,Once Upon a Time in America,8.2,1069,7.657811
883,640,Catch Me If You Can,7.7,3795,7.557097
281,4982,American Gangster,7.4,1502,7.141396
4041,11798,This Is England,7.4,363,6.739664
1149,168672,American Hustle,6.8,2807,6.717525
1243,203,Mean Streets,7.2,345,6.626569
2839,10220,Rounders,6.9,439,6.530427
